## Introduction & Project Context ##
In finance, options are contracts that give the buyer the right, not the obligation, to buy or sell an underlying asset at a strike price on or before the expiration date. Among many, the two most common styles of option contracts are European and American. For European-style options, the contract can only be exercised at expiration, so the Black-Scholes PDE can be applied to find a closed-form solution for the price of the option. However, American-style options can be exercised anytime before expiration, which introduces an early exercise boundary. For this reason, no straightforward PDE can be solved to find a formula for the price of the option. 

There are several techniques for attempting to price American options, but for this project, I will be focusing on the widely used Monte-Carlo method called the **Longstaff-Schwartz algorithm**. As machine learning models become more advanced in their predictive power, I will be exploring how different ML models can be used to perform the regression within the LS algorithm for pricing American options.



## How the Longstaff-Schwartz algorithm works ##

The LS algorithm has several parameters regarding the initial behavior of the stock and conditions of the option contract:

- $S_0 = \text{Intial stock price}$
- $r = \text{Risk-Free interest rate}$
- $\sigma = \text{Volatility}$
- $T = \text{Time to expiration}$
- $K = \text{Strike price}$
- $n_{trials} = \text{number of simulated paths}$
- $n_{timesteps} = \text{number of timesteps per path}$

This algorithm uses Monte Carlo simulations to generate different potential paths that the underlying asset may go through. It leverages a backward recursive approach and regression to estimate the expected value of the option. The expected value is compared to the current exercise value to find the optimal payoff at each timestep. The average of all the simulated option prices is taken to determine the final price of the option.


In [2]:
import numpy as np
import xgboost as xgb
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.pipeline import make_pipeline
from sklearn.neural_network import MLPRegressor

def longstaff_schwartz(S0, r, sigma, T, K, n_trials, n_timesteps, option_type, ml_model):
    """
    Core Longstaff-Schwartz implementation with ML models
    """
    # Monte-Carlo simulated paths
    S = np.zeros((n_trials, n_timesteps))
    S[:,0] = S0
    rng = np.random.default_rng(42)
    dW = rng.normal(0, 1, (n_trials, n_timesteps))
    dt = T / n_timesteps
    for t in range(1,n_timesteps):
        S_t = S[:,t-1] 
        S[:,t] = S_t + r*S_t*dt + sigma*S_t*dW[:,t-1] 

    
    ST = S[:,n_timesteps-1]
    payoffs = np.zeros((n_trials, n_timesteps))

    # Calculate terminal option prices
    if option_type == 'call':
        payoffs[:,n_timesteps-1] = np.maximum(ST - K, 0)
    elif option_type == 'put':
        payoffs[:,n_timesteps-1] = np.maximum(K - ST, 0)

    # Use backward induction to find final option price 
    for t in range(n_timesteps-2,-1,-1):
        # Determine which paths are in-the-money and out-of-the-money
        if option_type == 'call':
            itm_indices = np.where(S[:,t] > K)
            otm_indices = np.where(S[:,t] <= K)
        elif option_type == 'put':
            itm_indices = np.where(S[:,t] < K)
            otm_indices = np.where(S[:,t] >= K)
        # If no paths are in-the-money, do not exercise any option
        if len(itm_indices[0]) == 0:
            payoffs[:, t] = payoffs[:, t+1] * np.exp(-r*dt)
            continue
        X = S[itm_indices, t].reshape(-1, 1) # Current stock prices for in-the-money paths
        y = payoffs[itm_indices,t+1] * np.exp(-r*dt) # Discounted expected values of the option at the next timestep
        # Choose ML model
        match ml_model:
            case 'poly':
                model = make_pipeline(
                    PolynomialFeatures(degree=3),
                    LinearRegression(fit_intercept=False)
                )
                model.fit(X, y.ravel())
            case 'random forest':
                model = RandomForestRegressor()
                model.fit(X, y.ravel())
            case 'xgboost':
                model = xgb.XGBRegressor()
                model.fit(X, y)
            case 'mlp':
                model = MLPRegressor()
                model.fit(X, y.ravel())
            case _:
                raise ValueError(f"Unknown model type: {ml_model}")
            
        future_val = model.predict(X).flatten() # Predicted discounted expected value
        # Calculate current exercise price
        if option_type == 'call':
            current_val = np.maximum(S[itm_indices, t] - K, 0).flatten()
        elif option_type == 'put':
            current_val = np.maximum(K - S[itm_indices, t], 0).flatten()

        itm = itm_indices[0]  
        # Determines optimal time to exercise the option
        should_exercise = current_val > future_val
        # Sets payoffs accordingly
        payoffs[itm[should_exercise], t] = current_val[should_exercise]
        payoffs[itm[~should_exercise], t] = payoffs[itm[~should_exercise], t+1] * np.exp(-r*dt)
        payoffs[otm_indices, t] = payoffs[otm_indices, t+1] * np.exp(-r*dt)


    price = np.mean(payoffs[:,0]) # average of all option prices

    # 95% confidence interval
    SE = np.std(payoffs[:,0], ddof=1) / np.sqrt(n_trials)
    lower = price-1.96*SE
    upper = price+1.96*SE

    return price, lower, upper

## Models choice ##

In this project, I carefully chose four different machine learning approaches for the regression component within the Longstaff-Schwartz algorithm. For all models, the inputs are the stock prices for in-the-money paths only and the targets are discounted expected values of the option at the next timestep:

- **Polynomial Regression**
  - **Reason I chose this model:** This is the traditional baseline model that is typically implemented in the LS algorithm. It is computationally efficient, and should perform well in capturing the exercise boundary if it can be described by a polynomial.
  - **How it fits in the LS Algorithm:** At every timestep, the model tries to fit a polynomial of degree n for a function to find expected values of the option based on the current stock price. Predictions from this model come from the determined polynomial.
  
- **Random Forest**
  - **Reason I chose this model:** A random forest model could do a better job in this handling potential discontinuities that polynomial regression would not be able to handle. It can capture non-linear exercise boundaries without assuming an overall functional form.
  - **How it fits in the LS Algorithm:** At every timestep, the model creates several decision trees to categorize the expected value of the option based on the current stock price. It then averages the results from these trees for the final prediction.

- **XGBoost**
  - **Reason I chose this model:** This model could potentially be more efficient and robust than random forest. While both are ensemble algorithms, the sequential boosting in XGBoost could capture a lot more details that could be missed from the parallel bagging in random forest. Also, regularization will help model to avoid overfitting.
  - **How it fits in the LS Algorithm:** At every timestep, the model uses gradient boosting to sequentially add weak learners, each correcting previous errors to create a final robust learner. This final learner will be able to predict expected values of the option based on the current stock price. 

- **Multi-Layer Perceptron (MLP)**
  - **Reason I chose this model:** This model could theoretically outperform all other models given the correct hyperparameter tuning and regularization. Neural networks work well with large amounts of data, so using it in an algorithm that also uses Monte Carlo Simulation could have promising results.
  - **How it fits in the LS Algorithm:** At every timestep, layers of neurons are created, implicitly discovering its own relevant features and finding more nuanced patterns in the data. Then, using backpropagation, the MLP assigns weights to each neuron to create a universal function approximator capable of learning complex stock price to continuation value mappings.

For my benchmark, all ML-enhanced Longstaff-Schwartz results are compared against the binomial tree model, a widely accepted lattice-based pricing method that provides reliable American option valuations through backward induction.
